## Hardware-accelerated Reversi Board Detection on PYNQ Platform
2024.06.24 Naoki F., AIT. See LICENSE.txt for license information.

オーバーレイや必要なバッファを準備するためのクラスを宣言します．

Define a class for hardware overlay, required buffers, etc.

In [ ]:
from pynq import Overlay
from pynq import allocate
from pynq import PL
from PIL import Image
import numpy as np

# Accelerator and buffers in need
class ReversiAccelerator:
    def __init__(self, bitfile_name):
        PL.reset()              # make sure overlay cache is not used
        self.pl = Overlay(bitfile_name)
        self.accel = self.pl.reversi_accel_0
        self.rho   = allocate(shape=(32,), dtype=np.float32)
        self.theta = allocate(shape=(32,), dtype=np.float32)
        self.accel.write(0x1c, self.rho.device_address)   # rho_out
        self.accel.write(0x28, self.theta.device_address) # theta_out
        self.last_height = self.last_width = 0
        self.image = None
        
    def start(self, image):
        colored = (len(image.shape) == 3)
        if colored:
            height, width, _ = image.shape
        else:
            height, width = image.shape
        if width != self.last_width or height != self.last_height:
            if self.image is not None:
                self.image.freebuffer()
            if colored:
                self.image = allocate(shape=(height, width, 3), dtype=np.uint8, cacheable=1)
            else:
                self.image = allocate(shape=(height, width), dtype=np.uint8, cacheable=1)
            self.accel.write(0x44, width)  # stride
            self.accel.write(0x3c, width)  # width
            self.accel.write(0x34, height) # height
            self.last_width  = width
            self.last_height = height
        self.image[:] = image
        self.rho[:] = self.theta[:] = np.zeros(32)
        self.accel.write(0x10, self.image.device_address)
        self.accel.write(0x00, 1) # ap_start
        while (self.accel.read(0x00) & 0x02) == 0x00: # ap_done
            pass
        last_rho = last_theta = 0.0
        lines = []
        for i in range(32):
            if self.rho[i] == last_rho and self.theta[i] == last_theta:
                break
            lines.append([(self.rho[i], self.theta[i])])
            last_rho = self.rho[i]
            last_theta = self.theta[i]
        return lines

もしオリジナルのソフトウェアアルゴリズムをそのまま使いたい場合は，次のセルを実行します．

If you want to use the original software algorithm, run the following cell.

In [ ]:
accel = None
version = 1
my_eval = False

もしハフ変換を使ったアルゴリズムをソフトウェア実行したい場合は，次のセルを実行します．

If you want to use software implementation of the algorithm using Hough transform, run the following cell.

In [ ]:
accel = None
version = 1
my_eval = True

もしハフ変換だけをハードウェア実装したコアを使いたい場合は，次のセルを実行します．

If you want to use a hardware IP core that implements Hough transform only, run the following cell.

In [9]:
accel = ReversiAccelerator("reversi_hw_hough_only.bit")
version = 1
my_eval = True

In [ ]:
もし盤検出の一連のプロセスをハードウェア実装したコアを使いたい場合は，次のセルを実行します．

If you want to use a hardware IP core that implements a series of process of board detection, run the following cell.

In [7]:
accel = ReversiAccelerator("reversi_hw.bit")
version = 2
my_eval = True

どの場合でも，次のセルを実行すると，100枚の画像に対して盤認識を行い，その結果を表示します．

For any implementaion, run the following cell to perform board recognition to 100 images and display the results.

In [ ]:
import evaluate as ev

if 'accel' not in globals():
    accel = None # disable accelerator if not defined
if 'version' not in globals():
    version = 1
if 'my_eval' not in globals():
    my_eval = False
    
ev.eval_main(100, "images/answers.txt", accel=accel, my_eval=my_eval, version=version)